In [1]:
#submit this

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("/kaggle/input/cse-281-24-predict-the-item-price/train.csv")
df_test = pd.read_csv("/kaggle/input/cse-281-24-predict-the-item-price/test.csv")

df['X8'] = 2024 - df['X8']

replacement_mapping_X3 = {
    'LF': 'Low Fat',
    'low fat': 'Low Fat',
    'reg': 'Regular'
}
df['X3'] = df['X3'].replace(replacement_mapping_X3)
df_test['X3'] = df_test['X3'].replace(replacement_mapping_X3)

df['X2'].fillna(df['X2'].mean(), inplace=True)
df_test['X2'].fillna(df['X2'].mean(), inplace=True)

mode_X9 = df.pivot_table(values = 'X9', index = 'X11', aggfunc=(lambda x: x.mode()[0]))

missing_values = df['X9'].isnull()
df.loc[missing_values, 'X9'] = df.loc[missing_values, 'X11'].apply(lambda x: mode_X9.loc[x])
missing_values_test = df_test['X9'].isnull()
df_test.loc[missing_values_test, 'X9'] = df_test.loc[missing_values_test, 'X11'].apply(lambda x: mode_X9.loc[x])

df['X1'] = df['X1'].apply(lambda x: x[:2])
df_test['X1'] = df_test['X1'].apply(lambda x: x[:2])

df.replace({'X1': {'FD': 'Food', 'DR':'Drink','NC':'Non Consumable'}}, inplace = True)
df_test.replace({'X1': {'FD': 'Food', 'DR':'Drink','NC':'Non Consumable'}}, inplace = True)

le = LabelEncoder()

df['X9'] = le.fit_transform(df['X9'])
df_test['X9'] = le.transform(df_test['X9'])
df['X10'] = le.fit_transform(df['X10'])
df_test['X10'] = le.transform(df_test['X10'])
df['X11'] = le.fit_transform(df['X11'])
df_test['X11'] = le.transform(df_test['X11'])

columns_to_encode = ['X1', 'X3', 'X5', 'X7']
df = pd.get_dummies(df, columns=columns_to_encode, prefix_sep='_')
df_test = pd.get_dummies(df_test, columns=columns_to_encode, prefix_sep ='_')

df.drop('X5_Others', axis=1)

features = []
for col in df.columns:
    if df[col].corr(df['Y']) >= 0.2 or df[col].corr(df['Y']) <= -0.2:
        features.append(col)

features.remove('Y')
features_to_scale = ['X4', 'X6', 'X8','X9', 'X11']
scaler = MinMaxScaler()
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])
df_test[features_to_scale] = scaler.transform(df_test[features_to_scale])

X = df[features]
y = df['Y']
x_test = df_test[features]

svr_model = SVR(kernel='rbf', C=1000, epsilon=0.001)
svr_model.fit(X, y)

Y_pred = svr_model.predict(x_test)

submission = pd.DataFrame({'Y': Y_pred})
submission['row_id'] = range(submission.shape[0])
submission = submission[['row_id', 'Y']]
submission.to_csv('submission.csv', index=False)
print("Done!")

Done!
